### Excel Converersion of Dates from US to Euro Format

In [632]:
import pandas as pd
import os
import re
import openpyxl
from openpyxl.styles import Font, Fill, Color
from openpyxl.styles import colors
from openpyxl.styles import numbers
import numpy as np
np.set_printoptions(suppress=True)

from pathlib import Path
from datetime import datetime

In [633]:
path = 'C:/Users/bgeorgiev/Python Scripts - ipynb/ABS Converstion'
#path = Path.cwd()
#convertedpath = path+'/Converted'
#convertedpath = path /'Converted'
os.chdir(path)


In [634]:
# Define the diff function to show the changes in each field
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)

In [ ]:
def excel_diff(OLD, NEW, index_col):

    #df_OLD = pd.read_excel(path_OLD, index_col=index_col).fillna(0)
    #df_NEW = pd.read_excel(path_NEW, index_col=index_col).fillna(0)

    # Perform Diff
    dfDiff = df_NEW.copy()
    droppedRows = []
    newRows = []
    diffRows = []

    cols_OLD = df_OLD.columns
    cols_NEW = df_NEW.columns
    sharedCols = list(set(cols_OLD).intersection(cols_NEW))
    
    for row in dfDiff.index:
        if (row in df_OLD.index) and (row in df_NEW.index):
            for col in sharedCols:
                value_OLD = df_OLD.loc[row,col]

                value_NEW = df_NEW.loc[row,col]
                if value_OLD==value_NEW:
                    dfDiff.loc[row,col] = df_NEW.loc[row,col]
                else:
                    dfDiff.loc[row,col] = ('{}-->{}').format(value_OLD,value_NEW)
                    diffRows.append(row)
        else:
            newRows.append(row)

    for row in df_OLD.index:
        if row not in df_NEW.index:
            droppedRows.append(row)
            dfDiff = dfDiff.append(df_OLD.loc[row,:])

    dfDiff = dfDiff.sort_index().fillna('')
    #print(dfDiff)
    print('\nNew Rows:     {}'.format(newRows))
    print('Dropped Rows: {}'.format(droppedRows))

    # Save output and format
    fname = '{} vs {}.xlsx'.format(path_OLD.stem,path_NEW.stem)
    writer = pd.ExcelWriter(fname, engine='xlsxwriter')

    dfDiff.to_excel(writer, sheet_name='DIFF', index=True)
    df_NEW.to_excel(writer, sheet_name=path_NEW.stem, index=True)
    df_OLD.to_excel(writer, sheet_name=path_OLD.stem, index=True)

    diffRows = list(set(diffRows+newRows+droppedRows))
    df_Changes = dfDiff.loc[diffRows,:]
    print('\nChanges:\n{}'.format(df_Changes))

    # get xlsxwriter objects
    workbook  = writer.book
    worksheet = writer.sheets['DIFF']
    worksheet.hide_gridlines(2)
    worksheet.set_default_row(15)

    # define formats
    #date_fmt = workbook.add_format({'align': 'center', 'num_format': 'yyyy-mm-dd'})
    center_fmt = workbook.add_format({'align': 'center'})
    number_fmt = workbook.add_format({'align': 'center', 'num_format': '#,##0.00'})
    cur_fmt = workbook.add_format({'align': 'center', 'num_format': '$#,##0.00'})
    perc_fmt = workbook.add_format({'align': 'center', 'num_format': '0%'})
    
    #grey_fmt = workbook.add_format({'font_color': '#E0E0E0', 'diag_type': 2,'diag_color': 'red'})
    grey_fmt = workbook.add_format({'font_color': '#E0E0E0'})
    #drop_fmt = workbook.add_format({'font_color': 'red', 'bold':True})   
    drop_fmt = workbook.add_format({'font_color': 'white', 'bg_color': 'red'})
    #new_fmt = workbook.add_format({'font_color': '#32CD32','bold':True})
    new_fmt = workbook.add_format({'font_color': 'white','bg_color': 'green'})
    highlight_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})

    # set format over range
    ## highlight changed cells
    worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
                                            'criteria': 'containing',
                                            'value':'-->',
                                            'format': highlight_fmt})

#     ## highlight unchanged cells
#     worksheet.conditional_format('A1:ZZ1000', {'type': 'text',
#                                             'criteria': 'not containing',
#                                             'value':'-->',
#                                             'format': grey_fmt})
    # highlight new/changed rows
    #for row in range(dfDiff.shape[0]):
    for i,row in enumerate(dfDiff.index):
        #print(row, newRows)
        if row in newRows:
            print('new row', row)
            worksheet.set_row(i+1, 15, new_fmt)
        if row in droppedRows:
            print('dropped rows', row)
            worksheet.set_row(i+1, 15, drop_fmt)

    # save
    writer.save()
    print('\nDone.\n')

In [635]:
# import pyodbc
# from sqlalchemy.engine import create_engine
# config = { 'host': '10.25.31.117',
#                'user': 'sqluser',
#                'password': 'mis# Define the diff function to show the changes in each field
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)',
#                'database': 'ABSWeb_DaimlerUS' }
# user = 'sqluser'
# passw = 'mis'
# host =  '10.25.31.117'  # either localhost or ip e.g. '172.17.0.2' or hostname address 
# database = 'ABSWeb_DaimlerUS'

# engine = create_engine('mssql+pyodbc://' + user + ':' + passw + '@' + host + database , echo=False)
# #engine = create_engine('mysql+mysqlconnector://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

In [636]:
import pyodbc
server = '10.25.31.117'
databaseWeb = 'ABSWeb_DaimlerUS'
databaseNet = 'ABSDaimler_BUS'
username = 'sqluser'
password = 'mis'
con = 'Yes'
#Connection String
connectionWeb = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+databaseWeb+';UID='+username+';PWD='+ password)
connectionNet = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};SERVER='+server+';DATABASE='+databaseNet+';UID='+username+';PWD='+ password)



# cursor = connection.cursor()
# cursor.execute("SELECT * from DEAL_D;")
# row = cursor.fetchone()
# while row:
#     print(row[0])
#     row = cursor.fetchone()

In [637]:
_SQL = '''SELECT TRUST_ID, PCD_ACT_D, count(*) as Count, sum(S_invPrinBal) as S_InvPrinBal,  sum(S_invbegPrinBal) as S_InvBegPrinBal
from TRUCK_CS_MASTER
group by TRUST_ID, PCD_ACT_D
order by TRUST_ID, PCD_ACT_D
'''
indexcols = ['TRUST_ID','PCD_ACT_D']

_SQLWeb = _SQL
#_SQLWeb = 'Use [' + databaseWeb + '] '+ _SQL
_SQLNet = _SQL

print(_SQLWeb, _SQLNet)

SELECT TRUST_ID, PCD_ACT_D, count(*) as Count, sum(S_invPrinBal) as S_InvPrinBal,  sum(S_invbegPrinBal) as S_InvBegPrinBal
from TRUCK_CS_MASTER
group by TRUST_ID, PCD_ACT_D
order by TRUST_ID, PCD_ACT_D
 SELECT TRUST_ID, PCD_ACT_D, count(*) as Count, sum(S_invPrinBal) as S_InvPrinBal,  sum(S_invbegPrinBal) as S_InvBegPrinBal
from TRUCK_CS_MASTER
group by TRUST_ID, PCD_ACT_D
order by TRUST_ID, PCD_ACT_D



In [638]:
dfWeb = pd.read_sql_query(_SQLWeb, connectionWeb)
dfNet = pd.read_sql(_SQLNet, connectionNet)
connectionWeb.close()
connectionNet.close()

In [639]:
dfWeb['ID'] =dfWeb.loc[:,dfWeb.columns.isin(indexcols)].apply(lambda x: ' '.join(x.astype(str)), axis=1)
dfWeb.set_index('ID', inplace = True,drop = True) 
print('Web')
dfWeb.head()

Web


,TRUST_ID,PCD_ACT_D,Count,S_InvPrinBal,S_InvBegPrinBal
ID,,,,,
DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09
DTRT20181 2018-04-30 00:00:00,DTRT20181,2018-04-30,9522,1.028435e+09,1.066553e+09
DTRT20181 2018-05-31 00:00:00,DTRT20181,2018-05-31,9425,9.906638e+08,1.028435e+09
DTRT20181 2018-06-30 00:00:00,DTRT20181,2018-06-30,9294,9.456981e+08,9.906638e+08
DTRT20181 2018-07-31 00:00:00,DTRT20181,2018-07-31,9112,9.080982e+08,9.456981e+08


In [640]:
dfNet['ID'] =dfNet.loc[:,dfNet.columns.isin(indexcols)].apply(lambda x: ' '.join(x.astype(str)), axis=1)
dfNet.set_index('ID', inplace = True,drop = True) 
print('Net')
dfNet.head()

Net


,TRUST_ID,PCD_ACT_D,Count,S_InvPrinBal,S_InvBegPrinBal
ID,,,,,
DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09
DTRT20181 2018-04-30 00:00:00,DTRT20181,2018-04-30,9522,1.028435e+09,1.066553e+09
DTRT20181 2018-05-31 00:00:00,DTRT20181,2018-05-31,9425,9.906638e+08,1.028435e+09
DTRT20181 2018-06-30 00:00:00,DTRT20181,2018-06-30,9294,9.456981e+08,9.906638e+08
DTRT20181 2018-07-31 00:00:00,DTRT20181,2018-07-31,9112,9.080982e+08,9.456981e+08


In [641]:
filenameNet = 'Query_Web'
sheetNet = 'Web'
with pd.ExcelWriter(filenameWeb +'.xlsx') as writer:  # doctest: +SKIP
    dfWeb.to_excel(writer, sheet_name= sheetWeb)
    #dfWeb.to_excel(writer, sheet_name='Details')

In [642]:
filenameNet = 'Query_Net'
sheetNet = 'Net'
with pd.ExcelWriter(filenameNet +'.xlsx') as writer:  # doctest: +SKIP
    dfWeb.to_excel(writer, sheet_name= sheetNet)
    #dfWeb.to_excel(writer, sheet_name='Details')

#### Compare Excels

In [682]:
# Read in the two files but call the data old and new and create columns to track
# old = pd.read_excel('sample-address-1.xlsx', 'Sheet1', na_values=['NA'])
# new = pd.read_excel('sample-address-2.xlsx', 'Sheet1', na_values=['NA'])
Web = pd.read_excel(filenameWeb+'.xlsx', sheetWeb, na_values=['NA'])
Net = pd.read_excel(filenameNet+'.xlsx', sheetNet, na_values=['NA'])
Web['version'] = "Web"
Net['version'] = "Net"

In [683]:
# We use the account numbers as the keys to check what is added, dropped and potentially changed
# Using sets makes the deduping easy and we can use set operations to figure out groupings
web_ids_all = set(Web['ID'])
net_ids_all = set(Net['ID'])

dropped_ids = web_ids_all - net_ids_all
added_ids = net_ids_all - web_ids_all

In [684]:
#Join all the data together and ignore indexes so it all gets concatenated
all_data = pd.concat([Web,Net],ignore_index=True)

In [685]:
all_data

,ID,TRUST_ID,PCD_ACT_D,Count,S_InvPrinBal,S_InvBegPrinBal,version
0,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Web
1,DTRT20181 2018-04-30 00:00:00,DTRT20181,2018-04-30,9522,1.028435e+09,1.066553e+09,Web
2,DTRT20181 2018-05-31 00:00:00,DTRT20181,2018-05-31,9425,9.906638e+08,1.028435e+09,Web
3,DTRT20181 2018-06-30 00:00:00,DTRT20181,2018-06-30,9294,9.456981e+08,9.906638e+08,Web
4,DTRT20181 2018-07-31 00:00:00,DTRT20181,2018-07-31,9112,9.080982e+08,9.456981e+08,Web
5,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Net
6,DTRT20181 2018-04-30 00:00:00,DTRT20181,2018-04-30,9522,1.028435e+09,1.066553e+09,Net
7,DTRT20181 2018-05-31 00:00:00,DTRT20181,2018-05-31,9425,9.906638e+08,1.028435e+09,Net
8,DTRT20181 2018-06-30 00:00:00,DTRT20181,2018-06-30,9294,9.456981e+08,9.906638e+08,Net
9,DTRT20181 2018-07-31 00:00:00,DTRT20181,2018-07-31,9112,9.080982e+08,9.456981e+08,Net


In [686]:
#all_data['ID'].tolist()

In [739]:
#Save the changes to excel but only include the columns we care about
output_columns = list(all_data.columns.values)
print(output_columns)
changes_subset = output_columns
changes_subset.remove('version')
changes_subset =  [ele for ele in changes_subset if ele not in indexcols] 
print(changes_subset)

['ID', 'TRUST_ID', 'PCD_ACT_D', 'Count', 'S_InvPrinBal', 'S_InvBegPrinBal', 'version']
['ID', 'Count', 'S_InvPrinBal', 'S_InvBegPrinBal']


In [740]:
# x = all_data[all_data['ID']=='DTRT20181  2018-07-31 00:00:00']
# x.S_InvPrinBal = np.round(x.S_InvPrinBal,1)
# x
# xch = x.drop_duplicates(subset='S_InvPrinBal', keep='last')
# xch

In [741]:
# Let's see what changes in the main columns we care about
# Change drop_duplicates syntax: keep=last
#changes = all_data.drop_duplicates(subset=["ID"], keep='last')
changes = all_data.drop_duplicates(subset=changes_subset, keep='last')
changes

,ID,TRUST_ID,PCD_ACT_D,Count,S_InvPrinBal,S_InvBegPrinBal,version
0,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Web
5,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Net
6,DTRT20181 2018-04-30 00:00:00,DTRT20181,2018-04-30,9522,1.028435e+09,1.066553e+09,Net
7,DTRT20181 2018-05-31 00:00:00,DTRT20181,2018-05-31,9425,9.906638e+08,1.028435e+09,Net
8,DTRT20181 2018-06-30 00:00:00,DTRT20181,2018-06-30,9294,9.456981e+08,9.906638e+08,Net
9,DTRT20181 2018-07-31 00:00:00,DTRT20181,2018-07-31,9112,9.080982e+08,9.456981e+08,Net


In [742]:
#Get all the duplicate rows
dupe_ids = changes[changes['ID'].duplicated() == True]['ID'].tolist()
dupes = changes[changes["ID"].isin(dupe_ids)]
dupes

,ID,TRUST_ID,PCD_ACT_D,Count,S_InvPrinBal,S_InvBegPrinBal,version
0,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Web
5,DTRT20181 2018-03-31 00:00:00,DTRT20181,2018-03-31,9703,1.066553e+09,1.150009e+09,Net


In [743]:
# Pull out the old and new data into separate dataframes
change_Web = dupes[(dupes["version"] == "Web")]
change_Net = dupes[(dupes["version"] == "Net")]

In [744]:
# Drop the temp columns - we don't need them now
change_Web = change_Web.drop(['version'], axis=1)
change_Net = change_Net.drop(['version'], axis=1)

In [745]:
# Index on the account numbers
change_Web.set_index('ID', inplace=True)
change_Net.set_index('ID', inplace=True)

In [746]:
df_all_changes = pd.concat([change_Web, change_Net],
                           axis='columns',
                           keys=['Web', 'Net'],
                           join='outer')

In [747]:
df_all_changes

Web                                 \
                                  TRUST_ID  PCD_ACT_D Count  S_InvPrinBal   
ID                                                                          
DTRT20181  2018-03-31 00:00:00  DTRT20181  2018-03-31  9703  1.066553e+09   

                                                       Net                   \
                               S_InvBegPrinBal    TRUST_ID  PCD_ACT_D Count   
ID                                                                            
DTRT20181  2018-03-31 00:00:00    1.150009e+09  DTRT20181  2018-03-31  9703   

                                                              
                                S_InvPrinBal S_InvBegPrinBal  
ID                                                            
DTRT20181  2018-03-31 00:00:00  1.066553e+09    1.150009e+09

In [748]:
df_all_changes = df_all_changes.swaplevel(axis='columns')[change_Web.columns[0:]]

In [749]:
df_all_changes

TRUST_ID              PCD_ACT_D             \
                                       Web         Net        Web        Net   
ID                                                                             
DTRT20181  2018-03-31 00:00:00  DTRT20181   DTRT20181  2018-03-31 2018-03-31   

                               Count        S_InvPrinBal                \
                                 Web   Net           Web           Net   
ID                                                                       
DTRT20181  2018-03-31 00:00:00  9703  9703  1.066553e+09  1.066553e+09   

                               S_InvBegPrinBal                
                                           Web           Net  
ID                                                            
DTRT20181  2018-03-31 00:00:00    1.150009e+09  1.150009e+09

In [750]:
# xy = df_all_changes.groupby(level=0, axis=1)
# print(xy)

In [756]:
# Define the diff function to show the changes in each field
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} --> {}'.format(*x)

In [774]:
df_x = df_all_changes.groupby(level=0, axis=1)
# df_x.describe()

for key, item in df_x:
    print(df_x.get_group(key), "\n\n")

# gb = df_x.groups
# print(gb.keys())

# for key, value in gb.items():
#     print(key, value)
#     if key in key_list_from_gb:
#         print(df.ix[values], "\n")


                               Count      
                                 Web   Net
ID                                        
DTRT20181  2018-03-31 00:00:00  9703  9703 


                                PCD_ACT_D           
                                      Web        Net
ID                                                  
DTRT20181  2018-03-31 00:00:00 2018-03-31 2018-03-31 


                               S_InvBegPrinBal              
                                           Web           Net
ID                                                          
DTRT20181  2018-03-31 00:00:00    1.150009e+09  1.150009e+09 


                                S_InvPrinBal              
                                         Web           Net
ID                                                        
DTRT20181  2018-03-31 00:00:00  1.066553e+09  1.066553e+09 


                                  TRUST_ID            
                                       Web         Net
ID            

In [769]:
df_changed = df_all_changes.groupby(level=0, axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))
df_changed = df_changed.reset_index()

TypeError: invalid type promotion

In [758]:
df_changed = df_changed.reset_index()

In [759]:
df_changed

,index,ID,Net,Web
0,0,DTRT20181 2018-03-31 00:00:00,DTRT20181 --> 2018-03-31 00:00:00,DTRT20181 --> 2018-03-31 00:00:00


In [ ]:
df_removed = changes[changes["ID"].isin(dropped_ids)]
df_removed

In [ ]:
df_changed

In [ ]:
df_added = changes[changes["ID"].isin(added_ids)]
df_added

In [ ]:
print(output_columns)
print('\n')
print(df_changed.head(1))
print('\n')
print(df_removed.head(1))
print('\n')
print(df_added.head(1))

In [ ]:
#Save the changes to excel but only include the columns we care about
#output_columns = list(dfWeb.columns.values)

writer = pd.ExcelWriter("Diffs.xlsx")
df_changed.to_excel(writer,"changed", index=False, columns=output_columns)
df_removed.to_excel(writer,"removed",index=False, columns=output_columns)
df_added.to_excel(writer,"added",index=False, columns=output_columns)
writer.save()

In [ ]:
# # Get all the PDF filenames.
# ExcelFiles = []
# for filename in os.listdir('.'):
#     if filename.endswith('.xlsx') and filename.upper().find('EUR')<0:
#         #print(filename.upper().find('EUR'))
#         ExcelFiles.append(filename)
# ExcelFiles.sort()
# print('Number of source files in Folder:' , len(ExcelFiles))

In [ ]:
# for i, filename in enumerate(ExcelFiles):
#     print(i, 'Start: Converting:', filename, '...')
#     filename_noExt = (os.path.splitext(os.path.basename(filename))[0])
#     wb = openpyxl.load_workbook(filename)
#     for worksheet in wb.sheetnames:
#         print('...converting:', worksheet, 'tab')
#         ws = wb[worksheet] #wb.get_sheet_by_name(worksheet)
#         #row_count = sheet.max_row
#         #for i in range(row_count):
#         for row_cells in ws.iter_rows():
#             for cell in row_cells:
#                 if cell.value is not None:
#                     #print('orig:', cell.value)
#                     if FromUStoEuroDateMMDDYYYY(cell.value)!= None:
#                         #cell.value = FromUStoEuroDateMMDDYYYY(cell.value)
#                         cell.font = Font(color=colors.BLUE)
#                         cell.number_format = '*mm/dd/yyyy'
                    
#                     if FromUStoEuroDateYYYYMMDD(cell.value) != None:
#                         #cell.value = FromUStoEuroDateYYYYMMDD(cell.value)
#                         cell.font = Font(color=colors.BLUE)
#                         cell.number_format = '*mm/dd/yyyy'
                    
#                     if FromUStoEuroNumber(cell.value) != None:
#                         #cell.value = FromUStoEuroNumber(cell.value)
#                         cell.font = Font(color=colors.BLUE)
#                         cell.number_format ='0.00' #'General'
#                     #print('new:', cell.value)

#     convertedfilename = filename_noExt+'_{}.xlsx'.format('EUR')
#     wb.save(convertedpath+'/'+convertedfilename)
#     print(i,'End: File coverted to:',convertedfilename )
#     print('\n')
# print(i+1, 'Files Converted!!!')